**Run the following two cells before you begin.**

In [ ]:
%autosave 10

In [ ]:
import pandas as pd
import numpy as np

______________________________________________________________________
**First, import your data set and define the sigmoid function.**
<details>
    <summary>Hint:</summary>
    The definition of the sigmoid is $f(x) = \frac{1}{1 + e^{-X}}$.
</details>

In [1]:
# Import the data set
import pandas as pd

In [3]:
df = pd.read_csv('C:\\Users\\Lenovo\\cleaned_data.csv')

In [19]:
# Define the sigmoid function
def sigmoid(X):
    Y = 1 / (1 + np.exp(-X))
    return Y

**Now, create a train/test split (80/20) with `PAY_1` and `LIMIT_BAL` as features and `default payment next month` as values. Use a random state of 24.**

In [9]:
# Create a train/test split
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(df[['PAY_1','LIMIT_BAL']],df['default payment next month'],test_size=0.2,random_state=24)

______________________________________________________________________
**Next, import LogisticRegression, with the default options, but set the solver to `'liblinear'`.**

In [11]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='liblinear')

______________________________________________________________________
**Now, train on the training data and obtain predicted classes, as well as class probabilities, using the testing data.**

In [12]:
# Fit the logistic regression model on training data
lr.fit(x_train,y_train)

LogisticRegression(solver='liblinear')

In [25]:
# Make predictions using `.predict()`
lr_predict = lr.predict(x_test)
lr_predict

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [77]:
# Find class probabilities using `.predict_proba()`
y_proba = lr.predict_proba(x_test)
y_proba

array([[0.74826924, 0.25173076],
       [0.584297  , 0.415703  ],
       [0.79604453, 0.20395547],
       ...,
       [0.584297  , 0.415703  ],
       [0.82721498, 0.17278502],
       [0.66393435, 0.33606565]])

______________________________________________________________________
**Then, pull out the coefficients and intercept from the trained model and manually calculate predicted probabilities. You'll need to add a column of 1s to your features, to multiply by the intercept.**

In [18]:
# Add column of 1s to features
features = df[['PAY_1','LIMIT_BAL']]
features['constant'] = 1
features

<ipython-input-18-613f031cb1a0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['constant'] = 1


,PAY_1,LIMIT_BAL,constant
0,2,20000,1
1,-1,120000,1
2,0,90000,1
3,0,50000,1
4,-1,50000,1
...,...,...,...
26659,0,220000,1
26660,-1,150000,1
26661,4,30000,1
26662,1,80000,1


In [31]:
# Get coefficients and intercepts from trained model
print (lr.intercept_)
print (lr.coef_)

[-6.57647457e-11]
[[ 8.27451187e-11 -6.80876727e-06]]


### method-1

In [85]:
# Manually calculate predicted probabilities
V = x_test.values.dot(lr.coef_.transpose())
U = V + lr.intercept_
A = np.exp(U)
P =A/(1+A)
P /= P.sum(axis=1).reshape((-1, 1))
P

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]])

### method-2

In [43]:
from sklearn.utils.extmath import softmax
import numpy as np

scores = np.dot(x_train, lr.coef_.T) + lr.intercept_
softmax(scores)  # Sklearn implementation

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]])

______________________________________________________________________
**Next, using a threshold of `0.5`, manually calculate predicted classes. Compare this to the class predictions output by scikit-learn.**

In [83]:
# Manually calculate predicted classes
V = x_test.values.dot(lr.coef_.transpose())
U = V + lr.intercept_
A = np.exp(U)
P =A/(1+A)
P /= P.sum(axis=1).reshape((-1, 1))
pos = P>0.5
print(P[pos])
false = P<0.5
print(P[false])

[1. 1. 1. ... 1. 1. 1.]
[]


##### Compare to scikit-learn's predicted classes
when done manually, i got all predicted class as positive as u can see above.

______________________________________________________________________
**Finally, calculate ROC AUC using both scikit-learn's predicted probabilities, and your manually predicted probabilities, and compare.**

In [79]:
# Use scikit-learn's predicted probabilities to calculate ROC AUC
from sklearn import metrics
metrics.roc_auc_score(y_test,lr_predict)

0.5

In [84]:
# Use manually calculated predicted probabilities to calculate ROC AUC
metrics.roc_auc_score(y_test,P)

0.5

##### as you can see i got roc_auc_score =0.5 for both. 